In [11]:
from __future__ import annotations

from abc import ABC, abstractmethod
from typing import Generator, Optional, Callable, Any, Generic, TypedDict, Required, TypeVar, Literal
from pydantic.dataclasses import dataclass
from openai.types.responses import ResponseInputItemParam, ResponseFunctionToolCallParam
from openai.types.responses.response_output_message_param import ResponseOutputMessageParam
from openai.types.responses.response_reasoning_item_param import ResponseReasoningItemParam, Summary
from openai.types.responses.response_input_item_param import FunctionCallOutput
from enum import Enum
import json

MessageLike = ResponseInputItemParam

class StreamingEventType(Enum):
    REASONING_TOKEN = "reasoning_token"
    REASONING = "reasoning"
    TOKEN = "token"
    TOOL_CALL = "tool_call"
    TOOL_RESULT = "tool_result"
    COMPLETED = "completed"

class AgentStreamingEventType(Enum):
    REASONING = "reasoning"
    TOKEN = "token"
    ACTION = "action"
    ACTION_RESULT = "action_result"
    COMPLETED = "completed"


@dataclass
class ToolCall:
    name: str
    tool_call_id: str
    arguments: dict[str, Any]

@dataclass
class Context:
    conversation_id: str
    user_query: str
    messages: list[MessageLike]
    tools: set[Callable[..., str]]

@dataclass
class LLMStreamingEvent:
    event: StreamingEventType
    data: str
    tool_call_id: str | None = None
    tool_calls: list[ToolCall] | None = None

@dataclass
class AgentStreamingEvent:
    event: AgentStreamingEventType
    data: str
    action_id: str | None = None

class LLM(ABC):
    @abstractmethod
    def generate_text(self, messages: list[MessageLike], tools: Optional[set[Callable[..., str]]] = None) -> Generator[LLMStreamingEvent, None, None]:
        """Generate text based on the given prompt."""
        pass

class EmbeddingModel(ABC):
    @abstractmethod
    def embed_texts(self, texts: list[str]) -> list[list[float]]:
        """Generate embedding for the given texts."""
        pass

TIn = TypeVar("TIn")
TOut = TypeVar("TOut")
TNext = TypeVar("TNext")

class Handler(ABC, Generic[TIn, TOut]):
    @abstractmethod
    def process(self, input: TIn) -> TOut:
        pass

    def handle(self, input: TIn) -> TOut:
        return self.process(input)

    def then(self, next_handler: Handler[TOut, TNext]) -> Chain[TIn, TNext]:
        return Chain(self, next_handler)

class Chain(Handler[TIn, TOut]):
    """Represents two linked handlers as a single unit."""
    def __init__(self, first: Handler[TIn, TNext], second: Handler[TNext, TOut]):
        self.first = first
        self.second = second

    def process(self, input: TIn) -> TOut:
        intermediate = self.first.handle(input)
        return self.second.handle(intermediate)

T = TypeVar("T")

class Command(ABC, Generic[T]):
    @abstractmethod
    def exec(self, input: T) -> None:
        pass

class ToolHandler(Handler[ToolCall, str]):
    def __init__(
        self, 
        tools: set[Callable[..., str]],
        on_error: Optional[Callable[[ToolCall, Exception], str]] = None,
    ) -> None:
        self.tool_mappings: dict[str, Callable[..., str]] = {}
        self.add_tools(tools)
        self.on_error = on_error

    @property
    def tools(self) -> set[Callable[..., str]]:
        return set(self.tool_mappings.values())
    
    @property
    def tool_names(self) -> set[str]:
        return set(self.tool_mappings.keys())
    
    def clear_tools(self) -> None:
        """Remove all tools from the handler."""
        self.tool_mappings = {}
    
    def get_tool(self, name: str) -> Optional[Callable[..., str]]:
        return self.tool_mappings.get(name)
        
    def add_tools(self, tools: set[Callable[..., str]]) -> None:
        """Add tools to the handler."""
        for func_tool in tools:
            if func_tool.__name__ in self.tool_mappings:
                continue
            self.tool_mappings[func_tool.__name__] = func_tool

    def set_tools(self, tools: set[Callable[..., str]]) -> None:
        """Set the tools for the handler, replacing any existing tools."""
        self.clear_tools()
        self.add_tools(tools)

    def update_tool(self, tool: Callable[..., str]) -> None:
        """Update or add a single tool in the handler."""
        self.tool_mappings[tool.__name__] = tool

    def remove_tool(self, name: str) -> None:
        """Remove a tool by name from the handler."""
        if name in self.tool_mappings:
            del self.tool_mappings[name]

    def process(self, input: ToolCall) -> str:
        return self.execute_tool(input)

    def execute_tool(self, tool_call: ToolCall) -> str:
        tool = self.tool_mappings.get(tool_call.name)
        if not tool:
            return f"Error: Tool '{tool_call.name}' not found."

        try:
            return tool(**tool_call.arguments)
        except Exception as e:
            if self.on_error:
                return self.on_error(tool_call, e)
            else:
                return f"Error executing tool '{tool_call.name}': {str(e)}"
    
class ToolRouter(ToolHandler):
    """Base class for tool routing based on queries."""
    @abstractmethod
    def retrieve(self, query: str) -> set[Callable[..., str]]:
        """Retrieve appropriate tools based on the query."""
        pass

ToolUseBehavior = Literal[
    "stop_on_tool_call", 
    "stop_on_tool_result", 
    "auto"
]

ToolName = str

class Agent:
    def __init__(
        self, 
        llm: LLM, 
        tool_handler: ToolHandler,
        tool_use_behavior: Optional[ToolUseBehavior] = "auto",
        on_tool_call: Optional[dict[ToolName, Command[ToolCall]]] = None,
        on_tool_result: Optional[dict[ToolName, Command[tuple[ToolCall, str]]]] = None,
    ) -> None:
        super().__init__()
        self.llm = llm
        self.tool_handler = tool_handler
        self.tool_use_behavior = tool_use_behavior

        self.on_tool_call = on_tool_call
        self.on_tool_result = on_tool_result

    def add_tools(self, tools: set[Callable[..., str]]) -> None:
        """Add tools to the agent."""
        self.tool_handler.add_tools(tools)

    def run(self, messages: list[MessageLike]) -> Generator[LLMStreamingEvent, None, None]:
        """Run the agent with the given prompt."""
        while True:
            for delta in self.llm.generate_text(messages, tools=self.tool_handler.tools):
                if delta.event == StreamingEventType.REASONING_TOKEN:
                    yield delta
                if delta.event == StreamingEventType.REASONING:
                    yield delta
                    messages.append( 
                        ResponseReasoningItemParam( # type: ignore
                            type="reasoning",
                            summary=[Summary(
                                type="summary_text",
                                text=delta.data
                            )],
                        )
                    )
                elif delta.event == StreamingEventType.TOOL_CALL and delta.tool_calls:
                    yield delta
                    
                    for tool_call in delta.tool_calls:
                        messages.append(
                            ResponseFunctionToolCallParam(
                                type="function_call",
                                name=tool_call.name,
                                arguments=json.dumps(tool_call.arguments),
                                call_id=tool_call.tool_call_id,
                            )
                        )

                        if self.on_tool_call:
                            command = self.on_tool_call.get(tool_call.name)
                            if command:
                                command.exec(tool_call)
                    
                    if self.tool_use_behavior == "stop_before":
                        return
                    
                    for tool_call in delta.tool_calls:
                        tool_result = self.tool_handler.execute_tool(tool_call)
                        yield LLMStreamingEvent(
                            event=StreamingEventType.TOOL_RESULT,
                            tool_call_id=tool_call.tool_call_id,
                            data=tool_result,
                        )
                        messages.append(
                            FunctionCallOutput(
                                type="function_call_output",
                                output=tool_result,
                                call_id=tool_call.tool_call_id,
                            )
                        )

                        if self.on_tool_result:
                            command = self.on_tool_result.get(tool_call.name)
                            if command:
                                command.exec((tool_call, tool_result))
                        
                    if self.tool_use_behavior == "stop_after":
                        return
                elif delta.event == StreamingEventType.COMPLETED:
                    yield LLMStreamingEvent(
                        event=StreamingEventType.COMPLETED,
                        data=delta.data,
                    )
                    messages.append(
                        ResponseOutputMessageParam( # type: ignore
                            role="assistant",
                            content=delta.data
                        )
                    )
                    return
                elif delta.event == StreamingEventType.TOKEN:
                    yield LLMStreamingEvent(
                        event=StreamingEventType.TOKEN,
                        data=delta.data,
                    )
                
class ConversationRepository(ABC):
    @abstractmethod
    def get_conversation_history(self, conversation_id: str) -> list[MessageLike]:
        """Retrieve the conversation history for the given conversation ID."""
        pass

    @abstractmethod
    def add_messages(self, conversation_id: str, messages: list[MessageLike]) -> None:
        """Add messages to the conversation history."""
        pass

class ContextPrepareParam(TypedDict):
    conversation_id: Required[str]
    query: Required[str]

class ContextMiddleware(Handler[ContextPrepareParam, Context]):
    def __init__(
        self, 
        conversation_repository: ConversationRepository,
        tool_router: ToolRouter
    ) -> None:
        super().__init__()
        self.conversation_repository = conversation_repository
        self.tool_router = tool_router
        
    def process(self, input: ContextPrepareParam) -> Context:
        """Prepare context for the LLM based on conversation ID and query."""
        history = self.conversation_repository.get_conversation_history(input["conversation_id"])
        tools = self.tool_router.retrieve(input["query"])

        history.append({"role": "user", "content": input["query"]})
        
        return Context(
            conversation_id=input["conversation_id"],
            user_query=input["query"],
            messages=history,
            tools=tools
        )

class AgentService:
    def __init__(
        self, 
        agent: Agent,
        context_middleware: ContextMiddleware
    ) -> None:
        super().__init__()
        self.agent = agent
        self.context_middleware = context_middleware

    def handle_request(self, conversation_id: str, query: str) -> Generator[AgentStreamingEvent, None, None]:
        """Handle the request by preparing context and generating text."""
        context = self.context_middleware.process({"conversation_id": conversation_id, "query": query})
        self.agent.add_tools(context.tools)
        for delta in self.agent.run(context.messages):
            if delta.event == StreamingEventType.TOOL_CALL and delta.tool_calls:
                for tool_call in delta.tool_calls:
                    yield AgentStreamingEvent(
                        event=AgentStreamingEventType.ACTION,
                        data=f"Calling tool {tool_call.name} with arguments {tool_call.arguments}",
                        action_id=tool_call.tool_call_id or ""
                    )
            
            elif delta.event == StreamingEventType.TOOL_RESULT:
                yield AgentStreamingEvent(
                    event=AgentStreamingEventType.ACTION_RESULT,
                    data=delta.data,
                    action_id=delta.tool_call_id or ""
                )

            elif delta.event == StreamingEventType.COMPLETED:
                self.context_middleware.conversation_repository.add_messages(
                    conversation_id,
                    [{"role": "user", "content": query}, {"role": "assistant", "content": delta.data}]
                )
                yield AgentStreamingEvent(
                    event=AgentStreamingEventType.COMPLETED,
                    data=delta.data
                )

            else:
                yield AgentStreamingEvent(
                    event=AgentStreamingEventType.TOKEN,
                    data=delta.data
                )

In [12]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Fetches the current weather for a given city."""
    # Placeholder implementation
    return f"The current weather in {city} is sunny with a temperature of 75°F."

@tool
def get_info(user_name: str) -> str:
    """Fetches information about a user."""
    # Placeholder implementation
    return f"User {user_name} is a software developer from San Francisco."

model = ChatOpenAI(
    model="gpt-5.1",
    reasoning={"effort": "medium", "summary": "auto"},
    use_responses_api=True
)

agent = create_agent(
    model=model,
    tools=[get_weather, get_info],
    system_prompt="You're an assistant"
)

In [23]:
for event_type, (token, _) in agent.stream(
    # {
    #     "messages": [
    #         {"role": "user", "content": "Tell me the difference between MCP and Function Calling API"}
    #     ]
    # },
    {
        "messages": [
            {"role": "user", "content": "Tell me about the weather in New York and info about user Alice. Think carefully"}
        ]
    },
    stream_mode=["messages"]
):
    #print(token, type(token))
    delta_content = token.content
    if delta_content and isinstance(delta_content, list):
        # We can safely assume that there's only going to be 1 item within this list
        event = delta_content[0]
        print(event)


{'id': 'rs_0896681f62a5d5fb006926fd7ecd5881a393dfd54aa0556c2d', 'summary': [], 'type': 'reasoning', 'index': 0}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ''}], 'index': 0, 'type': 'reasoning', 'id': 'rs_0896681f62a5d5fb006926fd7ecd5881a393dfd54aa0556c2d'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': '**Planning'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ' parallel'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ' tool'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ' usage'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': '**\n\nThe'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ' user'}], 'index': 0, 'type': 'reasoning'}
{'summary': [{'index': 0, 'type': 'summary_text', 'text': ' is'}], 'index': 0, 'type': 'reasoning'}
{'